In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from scipy.io import wavfile
import librosa.display
import pyopenjtalk
from tacotron.frontend.openjtalk import text_to_sequence, numeric_feature_by_regex, pp_symbols
from tacotron import Tacotron2 
import pyaudio
import wave
import struct
import speech_recognition
from tqdm import tqdm
from IPython.display import Audio
from tacotron import tacotron2

In [36]:
sr = 16000

In [5]:
PATH = './my_model.pth'
model_params = torch.load(PATH)
model = Tacotron2()
model.load_state_dict(model_params)

<All keys matched successfully>

In [112]:
def wav2vec(filename):
    r = speech_recognition.Recognizer()
    with speech_recognition.AudioFile(filename) as source:
        audio_data = r.record(source)
    text = r.recognize_google(audio_data, language="ja-JP")
    print(text)
    labels = pyopenjtalk.extract_fullcontext(text)
    
    PP = pp_symbols(labels)
    in_feat = np.array(text_to_sequence(PP), dtype=np.int64)
    
    return in_feat

def mel2wav(x):
    n_fft = 2048
    frame_shift = 240
    sr = 16000
    gen_wav = librosa.feature.inverse.mel_to_audio(M=x.T, sr=sr, n_fft=n_fft, hop_length=frame_shift)
    return gen_wav

In [113]:

def voice2voice(record_second=2**3):
    # 音声のパラメータ
    CHUNK = 2**16 # チャンクサイズ
    RATE = CHUNK // record_second
    FORMAT = pyaudio.paInt16 # フォーマット
    CHANNELS = 1 # チャンネル数

    # pyaudioのインスタンスを作成
    p = pyaudio.PyAudio()

    # 入力用のデバイスのインデックス番号
    input_device_index = 0
    # 出力用のデバイスのインデックス番号
    output_device_index = 1

    # 入力用のストリームを開く
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    output=True,   
                      )

  
    in_data = stream.read(CHUNK)
    
    output_path = "./output.wav"
    wf = wave.open(output_path, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(in_data)
    wf.close()

    filename = "./output.wav"
    in_feat = wav2vec(filename)
    in_feat = torch.tensor(in_feat, dtype=torch.long)
    
    
    mel = model.inference(in_feat)[0].detach().numpy()
    wav = mel2wav(mel)
    wav = ((wav - wav.mean()) / wav.std())*(2**12)
    
    data_binary = wav.astype('int16').astype(np.int16).tobytes()
    #stream.write(data_binary)
    
    return wav

In [116]:
Audio(voice2voice(record_second=4), rate=sr)

おはようございます こんにちは


In [53]:
CHUNK = 2**16 # チャンクサイズ
RATE = CHUNK // 4
FORMAT = pyaudio.paInt16 # フォーマット
CHANNELS = 1 # チャンネル数

# pyaudioのインスタンスを作成
p = pyaudio.PyAudio()

# 入力用のデバイスのインデックス番号
input_device_index = 0
# 出力用のデバイスのインデックス番号
output_device_index = 1

# 入力用のストリームを開く
stream = p.open(format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                output=True,   
                  )


in_data = stream.read(CHUNK)
stream.write(in_data)

In [79]:
stream.write(data_binary)